In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Veriyi pandas'ın read_csv attribute'uyla okuyup csv'den dataframe'e çeviriyorum.

In [ ]:
diabetes=pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv",sep=",")

Verinin ilk beş satırına kolonlarına bakıyorum.

In [ ]:
diabetes.head()

Kolon isimlerini türkçeye çevirdim.

In [ ]:
diabetes=diabetes.rename(columns={"Pregnancies":"hamilelik","Glucose":"glukoz","BloodPressure":"Tansiyon","SkinThickness":"deri_kalinligi","BMI":"beden_kitle","DiabetesPedigreeFunction":"dpf","Age":"yas","Outcome":"diyabet"})

Veri setinde kayıp veri olup olmadığına bakıyorum.

In [ ]:
diabetes.isnull().values.any()

.describe()'la verinin tanımlayıcı istatistiklerine bakıyorum, minimum kolonunda tansiyonu, insulin ve glukoz değerleri 0 olan insanlar var, bunlara bir alt değer belirleyip veri setinden düşürüyorum.

In [ ]:
diabetes.describe()

In [ ]:
diabetes.drop(diabetes[diabetes.Tansiyon < 60].index, inplace=True)

In [ ]:
diabetes.drop(diabetes[diabetes.glukoz < 100].index, inplace=True)

In [ ]:
diabetes.drop(diabetes[diabetes.Insulin< 25].index, inplace=True)

Kontrol etmek için tanımlayıcı istatistiklere yeniden bakıyorum.

In [ ]:
diabetes.describe()

Lojistik regresyonu görselleştirmek adına glukoz'un cinsiyetlerdeki değişimini scatter plot'la görselleştirdim.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 

s=sns.relplot(x="glukoz",y="diyabet", data=diabetes, kind="scatter", hue="glukoz")
plt.show()

Kolonları arasındaki korelasyona seaborn'un heatmap'iyle bakıyorum. 

In [ ]:
sns.set_palette("PuRd")
correlation=diabetes.corr()
sns.heatmap(correlation)
plt.show()

MinMaxScaler'la veriyi normalize edicem.

In [ ]:
import pandas as pd    
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
diabetes_scaled = scaler.fit_transform(diabetes)


In [ ]:
diabetes_scaled = pd.DataFrame(diabetes_scaled, columns=diabetes.columns)
diabetes_scaled

In [ ]:
X=diabetes_scaled.loc[:,diabetes_scaled.columns!="diyabet"]
y=diabetes_scaled.diyabet

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
predrfc = rfc.predict(X_test)
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_test, predrfc))

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [{"n_estimators":[5, 10, 20, 50], "max_depth":[3, 5, 9, 12]}]
grid_search = GridSearchCV(rfc, param_grid, cv=5, scoring="accuracy")
grid_search.fit(X, y)
grid_search.best_params_

In [ ]:
grid_search.best_score_

In [ ]:
import pandas as pd    
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
diabetes_scaled = pd.DataFrame(scaler.fit_transform(diabetes),columns = diabetes.columns)
diabetes_scaled

Y'de (hedef kolon, tahmin etmek istediğimiz) İnsulin kolonu var, X'te (Y'yi tahmin etmek için kullandığımız kolonlar) ise kalanları. Train_test_split'le X'i ve Y'i training ve test için %20 oranla böldüm.

In [ ]:
X=diabetes_scaled.iloc[:, diabetes.columns!="Insulin"]
y=diabetes_scaled.iloc[:,diabetes.columns=="Insulin"]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

scikitlearn.linear_model'den Linear Regression'ı import ettim. Training verisini regresyon'da fit ettim (benim için bir doğru çizdi gibi düşünebilirsiniz) ardından test verisini fit ettiğim doğruya koyarak tahminde bulundum. Bu tahmini gerçek verideki y'nin test için ayırdığım kısmıyla karşılaştırıcam, bunun için sklearn.metrics'ten r2_score'u import ettim. Kötü bir r2 değeri geldi.

In [ ]:
from sklearn.linear_model import LinearRegression
linreg=LinearRegression(fit_intercept=True)
linreg.fit(X_train, y_train)
predict_linear=linreg.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score
print(r2_score(predict_linear,y_test))

Regresyonun katsayılarını aşağıda bulabilirsiniz. (ax+b'deki a ve b)

In [ ]:
print('Coefficients: \n', linreg.coef_)
print('Intercept: \n', linreg.intercept_)

Bu sefer insanların diyabet olup olmadığını tahmin edicem, bunun için lojistik regresyon kullanıcam. Diyabet kolonunu y'ye ayırdıktan sonra train_test_split yapıyorum.

In [ ]:
X=diabetes_scaled.iloc[:,:-1]
y=diabetes_scaled.iloc[:,-1]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

Training verisini modele fit ettim, ardından aynı test verisini tahminde bulunması için .predict()'le modele verdim.

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression()
logreg.fit(X_train,y_train)
pred=logreg.predict(X_test)

Modeli değerlendirmek için classification report'u kullanıyorum, diğer modellere göre en iyi accuracy değerini lojistik regresyon verdi.

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print(classification_report(y_test,pred,digits=2))

In [ ]:
Aynı problemi lineer regresyonda kursak ne olurdu?

In [ ]:
Aynı problemi lineer regresyonda kursak ne olurdu

In [ ]:
Aynı problemi lineer regresyonda kursak ne olurdu

In [ ]:
from sklearn.linear_model import LinearRegression
linreg=LinearRegression(fit_intercept=True)
linreg.fit(X_train, y_train)
predict_linear=linreg.predict(X_test)
print(r2_score(y_test,predict_linear))

Aynı problemi KNN'le çözelim.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5,metric='minkowski')
knn.fit(X_train,y_train)
predknn = knn.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print(classification_report(y_test,predknn,digits=2))

Veriyi support vector classifier'a verdim, kernel'i linear girdiğimde iyi bir sonuç aldım.

In [ ]:
from sklearn.svm import SVC
svclassifier=SVC(kernel="linear")
svclassifier.fit(X_train,y_train)
predsvc=svclassifier.predict(X_test)
print(classification_report(y_test,predsvc,digits=2))

Decision tree algoritması

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dec=DecisionTreeClassifier(criterion="entropy", max_depth=None)
dec.fit(X_train,y_train)
preddec = dec.predict(X_test)
print(classification_report(y_test,preddec,digits=2))

In [ ]:
Rassal orman algoritması

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(n_estimators=80, criterion='gini', max_depth=8)
rfc.fit(X_train,y_train)
predrfc=rfc.predict(X_test)
print(classification_report(y_test,predrfc,digits=2))

XGBoost

In [ ]:
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
xgbclassifier=XGBClassifier(silent=0, learning_rate=0.01, max_delta_step=5,
                            objective='reg:logistic',n_estimators=100, 
                            max_depth=3, eval_metric="logloss", gamma=3)
xgbclassifier.fit(X_train, y_train)
predxgb = xgbclassifier.predict(X_test)
print(classification_report(y_test,predxgb,digits=2))
print(accuracy_score(y_test, predxgb))